# ALI numerical algorithm for two-stream transfer

## Imports

In [ ]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

## 1) Set parameters, variables, and boundary values

### Set the grid

In [ ]:
# Set the number of grid points
n = 70

# Set the 1D grid
z = np.linspace(0.,1.,num=n)

# Set delta z (grid spacing value)
del_z = z[1]

### Gas variables

In [ ]:
# Set the thermal source function (aka the Planck function)
# In this example it is constant at each depth
B = np.ones(n)

# Photon destruction probability
# In this example it is constant 
epsilon = np.ones(n)*0.001

### Extinction coefficient

In [ ]:
# Set the extinction coefficient
def alpha(z):
    """
    Input: Depth z 
    Output: Extinction coefficient at the depth value given
    """
    
    return(10.**(5. - 6.*z)) 

# For the example problem this is 0
# but it will not always be 0

### Optical depth

In [ ]:
# Optical depth, (delta tau)
# Right now it's set based on example 4.4.5
# but there will be more options to 
# calculate this depending on what 
# information is given in the problem
# It is also constant here because our
# grid spacing is equal
def tau_fn(l,alpha_fn,delta_z,z_arr):
    """
    Input:
        n = number of grid points
        alpha_fn = Function for optical depth
    Output:
        tau = array of the optical depth at each grid point
    """
    
    tau_arr = np.zeros((l))
    for i in range(0,l):
        j = z_arr[i] + delta_z/2.
        tau_arr[i] = np.sqrt(3.)*alpha_fn(j)*delta_z
    return(tau_arr)

In [ ]:
tau = tau_fn(n,alpha,del_z,z)

## 2) Create interpolation functions and a matrix solver

### Quadratic Bezier Interpolation for I(U)

In [ ]:
# Interpolation coefficients 
# Only need interpolation if working in 2D or 3D
# Still not exactly sure how to define t
#t = 0.5
#m = 1. - t

#def I_interp(i,j):
#    """
#    Input: location on grid
#        i = x-coordinate 
#        j = y-coordinate
#    Output: Value of thrid-order-quadrature specific intensity value 
#        at the input location on grid
#    """
#    return(I[i-1,j]*u**2 + I[i+1,j]*2.*u*t + I[i+2,j]*t**2)

### Third order quadratic interpolation for the source contribution

In [ ]:
# Interpolation coefficients
e0 = np.ones((n)) - np.exp(-tau)
e1 = tau - e0
e2 = tau**2 - (2.*e1)

u_p = e0[0:-1] + (e2[0:-1] - 3.*tau[0:-1]*e1[0:-1])/(2.*tau[0:-1]**2)
p_p = ((2.*tau[0:-1]*e1[0:-1])-e2[0:-1])/(tau[0:-1]**2)
d_p = (e2[0:-1] - tau[0:-1]*e1[0:-1])/(2.*tau[0:-1]**2)

u_m = e0[1:n] + (e2[1:n] - 3.*tau[1:n]*e1[1:n])/(2.*tau[1:n]**2)
p_m = ((2.*tau[1:n]*e1[1:n])-e2[1:n])/(tau[1:n]**2)
d_m = (e2[1:n] - tau[1:n]*e1[1:n])/(2.*tau[1:n]**2)


# Define the 1D source term interpolation function
def S_interp(i,S,plus=True):
    """
    Input: 
        i = location on grid, make sure to not exceed i=n-2
        S = Source value at each gridpoint
        plus = True if we are dealing with the ray that
               is being integrated from the bottom to top
               False if integrated from the top to bottom
    Output: Value of thrid-order-quadrature source value
            for the two-stream approximation
        at the input location on grid
    """
    
    if plus==True:
        # This calculates u_(+,i+1/2), p_(+,i+1/2), d_(+,i+1/2)
        return(u_p[i]*S[i-1] + p_p[i]*S[i] + d_p[i]*S[i+1])
    elif plus==False:
        # This calculates u_(-,i+1/2), p_(-,i+1/2), d_(-,i+1/2)
        return(u_m[i]*S[i+1] + p_m[i]*S[i] + d_m[i]*S[i-1])

### Partial Lambda Operator

In [ ]:
# These are the lower diagonal,
# diagonal, and upper diagonal
# pieces of the simplified lambda
# operator. These will vary across 
# the grid if tau varies with z

v = len(u_p)

# Lower Diagonal 
lmbda_l = 0.5*(u_p+d_m)

# Diagonal 
lmbda_d = 0.5*(p_p+p_m)

# Upper Diagonal
lmbda_u = 0.5*(d_p+u_m)

## M matrix

In [ ]:
# M* = [Identity - (1-epsilon)*Lambda*]
# Where Lambda* is the partial lambda operator 
# The matrix equation that will be solved is:
# M*S = epsilon*B
#M = np.identity(n) - lmbda*(np.ones((n))-epsilon)


# Lower Diagonal of M
a =  -(np.ones((v))-epsilon[0:v])*lmbda_l

# Diagonal 
b = np.ones((v)) - (np.ones((v))-epsilon[0:v])*lmbda_d

# Upper Diagonal 
c =  -(np.ones((v))-epsilon[0:v])*lmbda_u

### Forward elimination and backward substitution subroutine

In [ ]:
# Define the functions for the coefficients
# used in the Thomas Algorithm which is used 
# and explained below
def gamma_beta_fn(a,b,c,y,n):
    """
    Input:
        a = upper diagonals of matrix M
        b = diagonals of matrix M
        c = lower diagonals of matrix M
        y = the solution array in the matrix equation: Mx = y (nx1)
        n = the number of rows in matrix m
    Output:
        g = the array of gamma constants (nx1)
        b = the array of beta constants (nx1)
    """
    
    # Define column arrays for necessary row-specific constants
    # This automatically sets gamma[0] = beta[0] = 0 which is 
    # necessary for the problem
    g = np.zeros((n))
    beta = np.zeros((n))
    
    for i in range(0,n-1):
            g[i+1] = (-c[i]/(a[i]*g[i] + b[i]))
            beta[i+1] = (y[i] - beta[i]*a[i])/b[i]
    
    return(g,beta)

In [ ]:
# Calculate Y = alpha_a(J - B) from MJ = alpha_a(J - B) = Y
Y = epsilon*B

# Compute the necessary gamma and beta constants
# This is essentially performing forward substitution 
gamma, beta_0 = gamma_beta_fn(a,b,c,Y,n)

In [ ]:
# Thomas Algorithm
# Forward elimination for a tridiagonal matrix
# Based on the algorithm found in Dullemond Ch4 notes
# Backward Substition is then applied
def tri_solver(S,n,B,gam,bet):
    """
    Input: 
        J = specific intensity at every grid point
        alpha_t = total opacity
        alpha_a = absorption opacity
        B = thermal source function
    Output:
        A new matrix or vector of J values 
    """
    
    #-----------------------------------------------------------
    # Perform forward elimination and backward substitution on M
    # Thomas Algorithm: a simplified form of Gaussian Elimination
    # specifically for tridiagonal matrices 
    
    # Use the gamma and beta constant arrays to compute 
    # what is essentially a back substitution to finish the
    # calculation of MX = Y
    X = np.zeros((n+1))
    X[0:n] = S
    for k in range(1,n):
        # Actually need to go from n, n-1, n-2,...,1
        # So create a new place holder that goes backwards
        c = n - k
        X[c] = gam[c]*X[c+1] + bet[c]
    
    # Return the new solution to J
    return(X[0:n])

## 3) Solve for converged S

In [ ]:
# Initialize I+ and I-
# These are not initial values
I_plus = np.zeros((n-1))
I_plus[0] = 1.
I_minus = np.zeros((n-1))

# Start with an initial guess for J
# The mean specific intensity
J = np.zeros((n))
J[0] = 1.

# Set the initial source function
S = epsilon*B + (np.ones(n) - epsilon)*J

for j in range(0,101):
    
    # Solve for I
    # I = exp(-tau)*I_prev + S_interp
    # ----------------------------------------------------------------------
    # For 2D and 3D use the function defined for 
    # quadratic Bezier interpolation to find the new value for I(U)
    # For 1D just use the i-1 grid point for the exponential term
    # Use a different third-order-quadrature method for S contributions
    for i in range(1,n-1):
        I_plus[i] = np.exp(-tau[i-1])*I_plus[i-1] + S_interp(i,S,plus=True)
        I_minus[i] = np.exp(-tau[i])*I_minus[i] + S_interp(i,S,plus=False)
        
        
    # Solve for J
    # J = 0.5*int_(-1)^(1) I(mu)*d mu
    # ----------------------------------------------------------------------
    J[1:] = 0.5*(I_plus + I_minus)
    
    # Solve for S
    # ----------------------------------------------------------------------
    # The 1D diffusion equation is a 2nd order PDE where lambda[S] is the solution
    # The numerical representation used for this PDE is Central Space
    # Forward substitution + backward substitution are used to solve the PDE
    S = epsilon*B + (np.ones(n) - epsilon)*J
    S = tri_solver(S,n,B,gamma,beta_0)
        
print(j)

In [ ]:
y = S/B
plt.semilogy(z,y)
plt.title("S/B vs z, epsilon=%1.3f" %epsilon[0])
plt.xlabel("z")
plt.ylabel("S/B")
plt.ylim(0.001,1.2)
plt.xlim(0.,1.0)
plt.show()